In [24]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt # for data visualization purposes
import seaborn as sns # for statistical data visualization
%matplotlib inline

In [25]:
data = './dataset/finalRoad.csv'
df = pd.read_csv(data)
df.head

<bound method NDFrame.head of        Accident_Index  longitude   latitude  Accident_Severity  Day_of_Week  \
0                   1  78.710393  14.724026                  2            3   
1                   2  78.534042  14.772353                  3            4   
2                   3  78.470877  14.745707                  3            5   
3                   4  78.557994  14.777128                  3            7   
4                   5  78.577431  14.703443                  3            2   
...               ...        ...        ...                ...          ...   
59993           59994  70.144710  24.587273                  3            4   
59994           59995  87.530231  25.803174                  3            4   
59995           59997  70.547520  25.773273                  3            4   
59996           59997  77.030311  25.308850                  3            4   
59997           59998  79.094879  21.735773                  3            4   

        Time  Speed_l

In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59998 entries, 0 to 59997
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Accident_Index      59998 non-null  int64  
 1   longitude           59998 non-null  float64
 2   latitude            59998 non-null  float64
 3   Accident_Severity   59998 non-null  int64  
 4   Day_of_Week         59998 non-null  int64  
 5   Time                59997 non-null  object 
 6   Speed_limit         59998 non-null  int64  
 7   Light_Conditions    59998 non-null  int64  
 8   Weather_Conditions  59998 non-null  int64  
 9   Date                59998 non-null  object 
dtypes: float64(2), int64(6), object(2)
memory usage: 4.6+ MB


In [27]:
df.isnull().sum()

Accident_Index        0
longitude             0
latitude              0
Accident_Severity     0
Day_of_Week           0
Time                  1
Speed_limit           0
Light_Conditions      0
Weather_Conditions    0
Date                  0
dtype: int64

In [28]:
X = df.drop(['Accident_Index','Accident_Severity', "Time", "Date"], axis = 1)
y = df['Accident_Severity']

In [29]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 2)

In [30]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors = 3)
knn.fit(X_train, y_train)
Y_pred = knn.predict(X_test)
acc_knn = round(knn.score(X_train, y_train) * 100, 2)
test_acc_knn = round(knn.score(X_test, y_test) * 100, 2)
# acc_knn
print('Train Score: ', acc_knn)  
print('Test Score: ', test_acc_knn) 

Train Score:  88.34
Test Score:  82.56


In [31]:
import urllib.request
import json
import time
from datetime import datetime
import paho.mqtt.publish as publish

In [32]:
channel_ID = "2061500"

mqtt_host = "mqtt3.thingspeak.com"

mqtt_client_ID = "PCEfJSM9Cwo3BSYULiwnNCw"
mqtt_username  = "PCEfJSM9Cwo3BSYULiwnNCw"
mqtt_password  = "OlAVymnEWY4mTMtfc5X5p37F"

t_transport = "websockets"
t_port = 80

topic = "channels/" + "2061500" + "/subscribe"

In [33]:
READ_API_KEY='CG50D1N7SZQ4ETMZ'
CHANNEL_ID= '2061500'
# global TS

In [34]:
try:
    # for i in range(10):
    while True:
        url = "http://api.thingspeak.com/channels/{}/feeds/last.json?api_key={}&fields=1,2,3".format(CHANNEL_ID, READ_API_KEY)
        response = urllib.request.urlopen(url)
        data = json.loads(response.read())
        
        url = "http://api.thingspeak.com/channels/{}/fields/1/last.json?api_key={}".format(CHANNEL_ID, READ_API_KEY)
        response = urllib.request.urlopen(url)
        data_field1 = json.loads(response.read())

        # Get data for field 2
        url = "http://api.thingspeak.com/channels/{}/fields/2/last.json?api_key={}".format(CHANNEL_ID, READ_API_KEY)
        response = urllib.request.urlopen(url)
        data_field2 = json.loads(response.read())

        # Get data for field 3
        url = "http://api.thingspeak.com/channels/{}/fields/3/last.json?api_key={}".format(CHANNEL_ID, READ_API_KEY)
        response = urllib.request.urlopen(url)
        data_field3 = json.loads(response.read())

        # Get data for field 1
        url = "http://api.thingspeak.com/channels/{}/fields/4/last.json?api_key={}".format(CHANNEL_ID, READ_API_KEY)
        response = urllib.request.urlopen(url)
        data_field4 = json.loads(response.read())

        # Get data for field 2
        url = "http://api.thingspeak.com/channels/{}/fields/5/last.json?api_key={}".format(CHANNEL_ID, READ_API_KEY)
        response = urllib.request.urlopen(url)
        data_field5 = json.loads(response.read())

        # Get data for field 3
        url = "http://api.thingspeak.com/channels/{}/fields/6/last.json?api_key={}".format(CHANNEL_ID, READ_API_KEY)
        response = urllib.request.urlopen(url)
        data_field6 = json.loads(response.read())
        
        a = data['created_at'][:10]
    # print (a)
        date_obj = datetime.strptime(a, '%Y-%m-%d').date()
        day = date_obj.isoweekday()
        day
        
        
        accident = []
        accident.append(float(data_field3["field3"]))
        accident.append(float(data_field4["field4"]))
        accident.append(day)
        accident.append(float(data_field1["field1"]))
        light = int(data_field2["field2"])
        if light >1000:
            accident.append(7)
        elif light < 50:
            accident.append(1)
        else:
            accident.append(4)

        if int(data_field2["field2"]) >500 and float(data_field2["field2"])<1000:
            accident.append(7)
        elif int(data_field6['field6']) == 0 and float(data_field5["field5"])<=0:
            accident.append(8)
        elif data_field6['field6'] == 1 and float(data_field5["field5"])<=0:
            accident.append(3)
        elif int(data_field6['field6']) == 0 and float(data_field5["field5"])>0 and float(data_field5["field5"])<20:
            accident.append(6)
        elif float(data_field5["field5"])>10 and float(data_field5["field5"])<35:
            accident.append(5)
        elif float(data_field5['field5']) == 0:
            accident.append(9)
        else:
            accident.append(1)
            
        accident = np.array(accident)
        a = knn.predict(accident.reshape(1, -1))
        print (a[0])

        channel_ID = "2061500"

        mqtt_host = "mqtt3.thingspeak.com"

        # Your MQTT credentials for the device
        mqtt_client_ID = "PCEfJSM9Cwo3BSYULiwnNCw"
        mqtt_username  = "PCEfJSM9Cwo3BSYULiwnNCw"
        mqtt_password  = "OlAVymnEWY4mTMtfc5X5p37F"

        t_transport = "websockets"
        t_port = 80

        topic = "channels/" + "2061500" + "/publish"
        payload = "field7=" + str(a[0])
        try:
            publish.single(topic, payload, hostname=mqtt_host, transport=t_transport, port=t_port, client_id=mqtt_client_ID, auth={'username':mqtt_username,'password':mqtt_password})

        except Exception as e:
            print (e) 
except:
    print("\n\nTHANKS, VISIT AGAIN!!")

c:\Python39\lib\site-packages\sklearn\base.py:409: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(


3


c:\Python39\lib\site-packages\sklearn\base.py:409: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(


3


c:\Python39\lib\site-packages\sklearn\base.py:409: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(


...

3


c:\Python39\lib\site-packages\sklearn\base.py:409: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(


3


c:\Python39\lib\site-packages\sklearn\base.py:409: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(


3


THANKS, VISIT AGAIN!!
